In [21]:
import os
import sys

# Add source folder
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../src/processing/local-geocode"))

# Load constants
from src.constants import DATA_PATH

# Using Geocode

In [22]:
from geocode.geocode import Geocode

geocoder = Geocode()
geocoder.init()

## single cpu

In [23]:
geocoder.decode("zurich")

## multi cpu

In [72]:
locations = ["zurich", "basel", "sarnen", "giswil", "goldach", "gruet", "bregenz"]
geocoder.decode_parallel(locations, num_cpus=4)

2020-09-19 10:44:15,873 [INFO ] [geocode.geoc]: Running decode in parallel with 4 cores
100%|██████████| 4/4 [00:00<00:00, 30.58it/s]


[[{'name': 'Zurich',
   'country_code': 'CH',
   'longitude': 8.66667,
   'latitude': 47.41667}],
 [{'name': 'Basel',
   'country_code': 'CH',
   'longitude': 7.593610000000001,
   'latitude': 47.55773}],
 [{'name': 'Sarnen',
   'country_code': 'CH',
   'longitude': 8.17646,
   'latitude': 46.89845}],
 [{'name': 'Giswil',
   'country_code': 'CH',
   'longitude': 8.118839999999999,
   'latitude': 46.83586}],
 [{'name': 'Goldach',
   'country_code': 'CH',
   'longitude': 9.46787,
   'latitude': 47.47364}],
 [{'name': 'Gruet',
   'country_code': 'CH',
   'longitude': 8.783389999999999,
   'latitude': 47.31151}],
 [{'name': 'Bregenz',
   'country_code': 'AT',
   'longitude': 9.919889999999999,
   'latitude': 47.42069}]]

# Get list of all places in Switzerland

In [61]:
import joblib
geonames = joblib.load("/tmp/local_geocode/geonames.pkl")

places_in_switzerland = []
for place in geonames:
    if place[1] == "CH":
        places_in_switzerland.append(place)

swiss_places = {entry[0] : tuple(entry[1:]) for entry in places_in_switzerland}
global_places = {entry[0] : tuple(entry[1:]) for entry in geonames}

# Save
#joblib.dump(swiss_places, "/mnt/data/location_data/swiss_places.pkl")
#joblib.dump(global_places, "/mnt/data/location_data/global_places.pkl")